In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

import Pygor
import datetime
from scipy.spatial.distance import cdist
from skimage.feature import blob_dog, blob_log
from VAE import FactorVAE
import cv2
import tensorflow as tf
import time

##do_2d varying c5_c9

# loading 2 VAEs            
def load_vae():
    # centered vae
    tf.reset_default_graph()
    vae = FactorVAE(gamma=1,alpha=34,bs=96)
    vae.load_latest_checkpoint(path='/media/oxml/DA12F77512F754CB/VAE_results/sub_test/variance_factorvae_bias_bs96_gam_1_a_34_20190814-170308/checkpoints')
        
    # augmented (translated images) vae
    tf.reset_default_graph()
    vae2 = FactorVAE(gamma=1,alpha=34,bs=96)
    vae2.load_latest_checkpoint(path='/media/oxml/DA12F77512F754CB/VAE_results/sub_test/variance_translated_factorvae_bias_bs96_gam_1_a_34_20190814-151632/checkpoints')
        
    return vae,vae2

# alternative do2d function
# ideal for center of trinagle location
# starting gate voltages (starting: c5_c9: center of triangle)
# future change to input probably dictionary: key:value
# BIAS("C1") SET TO 20 - depends on multipler on Delft 20 in pygor, 10m multuiplier --> 0.2mV bias
def do_2d(starting_gates=[-1159.9539658,-808.79439154,-1815.3845539,-1127.79332466,-1406.53212482,-682.98769082]
    ,starting_c5_c9=[-572,-943],window=200,res=40): # current map has span of <window> in mV in each dimension, <res> number of points

    # BIAS("C1") SET TO 20
    pygor.setvals(["c1","c3","c4","c6","c7","c10"],[20]+starting_gates)    
    pygor.setvals(["c5","c9"],starting_c5_c9)

    data=pygor.do2d("c9",starting_c5_c9[1]-window/2,starting_c5_c9[1]+window/2,res,"c5",starting_c5_c9[0]-window/2,starting_c5_c9[0]+window/2, res)
    data=data.data[0]

    xrange=[starting_c5_c9[0]-window/2,starting_c5_c9[0]+window/2]
    yrange=[starting_c5_c9[1]-window/2,starting_c5_c9[1]+window/2]
    
    #create dictionary to store resulting 2d plot, x&y axis range in voltage space, starting gates, 
    # N.B doesn't include scanning gates
    # Nina: dicttionary should store list of ALL gates as well as their corresponding values
    d={}
    d['img']=data
    d['xrange']=xrange
    d['yrange']=yrange
    d['gates']=starting_gates
    #d['scanning_gates'] = 

    return d,data

def random(savedir, gates_init, starting_c5_c9_init,vae,vae2,n_walk=10,it=30, window_small=30,res_low=30,res_high=60,res_save=128):
    
    time_list=[]
    
    for i in range(n_walk):
        
        # define lists for data storage
        gates_prop_save=[] # list of list proposed gate voltage values
        c5c9_prop_save=[] # list of list proposed scanning gate voltage values
        gates_accept=[] # accepted gates voltages where the score improves
        score_save=[] # score values
        high_res_save=[] # If the score  improves, we take a high res scan a save here -- good for publishing
        
        gates=gates_init
        print('gates:',gates)
        c5c9_list=starting_c5_c9_init
        print('init c5c9:', c5c9_list)
        
        dv=2 # grid step for each gate exceot the scanning ones
        # <np.array([index,-dv]),> index corresponds from index in gates=gate_init
        idx_init=[np.array([0,-dv]),np.array([0,dv]),np.array([1,-dv]),np.array([1,dv]),np.array([2,-dv]),np.array([2,dv]),np.array([3,-dv]),np.array([3,dv]),np.array([4,-dv]),np.array([4,dv])]
        idx=idx_init.copy()
        
        idx_tot=[]
        gates_tot=[]
        gates_tot.append(gates_init)
        # initial high res scan
        _,data_init=do_2d(starting_gates=gates,starting_c5_c9=c5c9_list, window=window_small,res=res_high)
        score_vae_old,score_vae_trans_old,score_tot_old=predict_mod(vae,vae2,data_init)
        
        data_save,_=do_2d(starting_gates=gates,starting_c5_c9=c5c9_list, window=window_small,res=res_save)
        data_save['score_init']=score_tot_old
        data_save['it']=0
        high_res_save.append(data_save)
        
        score_save.append(score_tot_old)
        gates_accept.append(gates)
        
            
        for j in range(it):
            
            t_init=time.time()
            print('Search %d, step %d' %(i+1,j+1))
                  #get new gate voltages      
            gates_prop,delta_v, idx,gate_idx, idx_tot, gates_tot,gates=get_new_param(gates,gates_tot, idx,idx_tot)
            # everything finito - no options left
            if len(gates_prop)==0:
                print('run %d has terminated at iteration %d' %(i+1,j+1))
                break
                                
            print('new gates:', gates_prop)
            # large window for blob detection, 0.8 hardcoded factor, 
            window_large=window_small+0.8*abs(delta_v)*window_small/10
            
            t_0=time.time()

            print('large window size:', window_large)
            _,data_low=do_2d(starting_gates=gates_prop, starting_c5_c9=c5c9_list, window=window_large,res=res_low)          
            t_low=time.time()
            print(t_low-t_0)
            time_list.append(t_low-t_0)
            method='blob'
            if method=='blob':
                # return the new coordinates for c5 and c9 - this is why we keep them in a separate list to the rest of the gate
                c5c9_list_prop=blob_detection(data_low,c5c9_list,res_low=res_low,window_large=window_large)
            # Nina never uses maximum - always 'blob'                        
            if method=='maximum':
                c5c9_list_prop=find_max(data_low,c5c9_list,window_large=window_large,res_low=res_low)
            t_0=time.time()     
            _,data_high=do_2d(starting_gates=gates_prop,starting_c5_c9=c5c9_list_prop, window=window_small,res=res_high)
            t_high=time.time()
            print(t_high-t_0)
            time_list.append(t_high-t_0)
            score_vae,score_vae_trans,score_tot=predict_mod(vae,vae2, data_high)
             
            gates_prop_save.append(gates_prop)
            c5c9_prop_save.append(c5c9_list_prop)
            score_save.append(score_tot)
            
            # if new score is less (because it's distance fam) than initial scan score
            if score_tot<score_tot_old:
                # proposed gates become the new gates
                gates=gates_prop.copy() #accept if score is better
                gates_tot.append(gates)
                idx_tot.append(idx)
                idx=idx_init.copy()
                idx=[x for x in idx if not np.array_equal(x,np.array([gate_idx,-delta_v]))] # remove the option of directly going backwards
                
                c5c9_list=c5c9_list_prop.copy()                
                print('Voltage change accepted')
                score_tot_old=score_tot.copy()
                
                # A VERY HIGH RES SCAN FOR PUBLICATIONS
                #data_save,_=do_2d(starting_gates=gates,starting_c5_c9=c5c9_list, window=window_small,res=res_save)
                #data_save['score']=score_tot
                #data_save['it']=j
                #high_res_save.append(data_save)
                gates_accept.append(gates)
                
            t_f=time.time()
            time_dict={j,(t_f-t_init)}
            time_list.append(time_dict)
        
            print('c5c9: ', c5c9_list)
            #date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  
            #np.save('/home/oxml/Dropbox/deep_rl_quantum/2d_scans/random_search/random_search_c5c9_%s' %date, dic)   
            
            print('idx_tot: ', idx_tot)
            print('gates_tot: ', gates_tot)
            print('idx: ', idx)
            print('gates: ', gates)
            
    
        np.save('%stime_%d' %(savedir,i), time_list)
        np.save('%sgates_prop_%d'%(savedir,i),gates_prop_save)
        np.save('%sc5c9_prop_%d'%(savedir,i),c5c9_prop_save)
        np.save('%sscore_%d'%(savedir,i),score_save)
        #np.save('%shigh_res_%d'%(savedir,i),high_res_save)
    
# We are now in VAE land            
def predict_mod(vae,vae2, data):
    
    data_res=cv2.resize(data, dsize=(32,32), interpolation=cv2.INTER_LINEAR)   
    
    data_res=normalise(data_res)
    
    data_exp=data_res[np.newaxis,:,:, np.newaxis] # needs extra 2-d axis because VAE only takes 4d data

    #VAE score
    score_vae=vae.score(data_exp)
    score_vae_trans=vae2.score(data_exp)
    
    #Total score
    score_tot=(0*score_vae)+(1*score_vae_trans)
    print('score_vae: ', score_vae, 'score_vae_trans: ', score_vae_trans, 'tot_score: ', score_tot)
            
    return score_vae,score_vae_trans,score_tot
        
def get_new_param(gates, gates_tot, idx, idx_tot):
    #idx is a list of the gate indices combined with the voltage change (+dv or -dv)
    
    if len(idx)==0:
        if len(idx_tot)==0:
            print('The initial starting point and step size did not result in any score increase')
            return [],[],[],[],[],[],[] # number of empty lists should be equal to number of variables output by function
        
        idx_new=[]
        while len(idx_new)==0:
            if len(idx_tot)==0:
                print('The initial starting point and step size did not result in any score increase')
                return [],[],[],[],[],[],[] # number of empty lists should be equal to number of variables output by function

            idx_new=idx_tot[-1].copy()
            gates_new=gates_tot[-2].copy()
            gates=gates_tot[-2].copy()
            print('step back in voltage space')
            
            del idx_tot[-1]
            del gates_tot[-1]
            
    else:
        idx_new=idx.copy()
        gates_new=gates.copy()
    
    chosen=np.random.choice(np.arange(len(idx_new)))
        
    gate_idx, delta_v_new=idx_new[chosen][0],idx_new[chosen][1] 

    gates_new[gate_idx]=gates_new[gate_idx]+delta_v_new
    gate_str=["c3","c4","c6","c7","c10"] # hardcoded look north
    print('gate %s changes with %fmV' %(gate_str[gate_idx],delta_v_new))
    
    del idx_new[chosen]
    
    return gates_new, delta_v_new, idx_new, gate_idx, idx_tot,gates_tot, gates

# hardcoded normalisation factors - can change to max current value from Dom's high res scans maybe
def normalise(data,min_I=-1.94e-10,norm_range=150e-12):
        
    img_norm=(data-min_I)/(norm_range)
    img_norm=np.clip(img_norm, 0, 1) #clip so that values outside range are within 0,1
    
    plt.figure()
    plt.imshow(img_norm, vmin=0,vmax=1)
    plt.colorbar()
    plt.show()
    
    return img_norm
    
def blob_detection(image,c5c9_list,window_large,res_low):
    
    delta_triangle=3 #min mV dimension of a triangle
    delta_triangle_max=15 #max mV dimension of a triangle
    #alpha=50 #makes sure no noise is detected
    
    min_sigma=delta_triangle*res_low/window_large
    max_sigma=delta_triangle_max*res_low/window_large
    
    range_data=[np.min(image),np.max(image)]
    norm=(image-range_data[0])/(range_data[1]-range_data[0])*255
    
    #thresh=np.mean(np.sort(norm.flatten())[:int(norm.shape[0])])*alpha
    thresh=30
    #print(np.mean(np.sort(norm.flatten())[:int(norm.shape[0])]))
    print('min_sigma: ', min_sigma, 'max_sigma: ', max_sigma, 'thres: ', thresh)
    # blob_log(image, mimimum blob size you want to detect, max blob size you want to detect, threshold=mim current value for something to be considered in blob detection)
    # blob_log = Laplacian of Gaussian
    # blob_dog = difference of Gaussian -= often fails so we don't use
    blobs = blob_log(norm, min_sigma=min_sigma, max_sigma=max_sigma, threshold=thresh)
    blobs_2= blob_dog(norm, min_sigma=min_sigma, max_sigma=max_sigma, threshold=thresh)
    
    print('log', blobs)
    print('dog', blobs_2)
    
    xa=np.array([[res_low/2,res_low/2]]) #Pixelwise center of 2d scan
    
    if len(blobs)>1:
        idx_min=np.argmin(cdist(xa,blobs[:,:2])) #Select detected feature which is closest to center
        coor=blobs[idx_min][:2]
    elif len(blobs)==1:
        coor=blobs[0][:2]
        
    elif len(blobs)==0:
        return c5c9_list
    
    print('coor: ', coor)
    diff=xa[0]-np.array(coor)
    # converting pixels to voltage space
    c5_new=c5c9_list[0]-diff[1]*window_large/res_low
    c9_new=c5c9_list[1]-diff[0]*window_large/res_low
    c5c9_list=[c5_new,c9_new]
                
    print('c5_c9: ', c5c9_list)
    
    return c5c9_list
                    
def find_max(data,c5c9_list, window_large,res_low):
    coor=np.unravel_index(data.argmax(), data.shape)
    print(coor)
    diff=np.array([res_low/2,res_low/2])-np.array(coor)
    print(diff)

    c5_new=c5c9_list[0]-diff[1]*window_large/res_low
    c9_new=c5c9_list[1]-diff[0]*window_large/res_low

    c5c9_list=[c5_new,c9_new]

    return c5c9_list


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
--------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Welcome to Pygor!
--------------------------------------------------
To start the server:  pygor = Experimentfrontend()
New experiment created in: pygor.savedir

To take measurements use functions 
pygor.do2d, pygor.do1d and pygor.do0d.
To control use pygor.setval and pygor.getval.
Function syntax is identical to Igor!
--------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [2]:
def run_random(savedir, window_small=30, res_low=10, res_high=20, n_walk=30,it=10, window_large_init=30,
               gates_init=[-1150.9539658,-808.79439154,-1815.3845539,-1127.79332466,-1406.53212482,-682.98769082],
               starting_c5_c9_init=[-575,-945],res_save=128):
    
    
    vae,vae2=load_vae()

    #Do an initial low res 2d scan to determine center c5 and c9
    _,data_init_low=do_2d(starting_gates=gates_init, starting_c5_c9=starting_c5_c9_init, window=window_large_init,res=res_low)
    c5c9_list_init=blob_detection(data_init_low,c5c9_list=starting_c5_c9_init,res_low=res_low,window_large=window_large_init)
                      
    #Run the random search with this c5 and c9 as new center
    random(gates_init=gates_init,starting_c5_c9_init=c5c9_list_init,vae=vae,vae2=vae2,n_walk=n_walk,it=it,window_small=window_small,res_low=res_low,res_high=res_high,savedir=savedir,res_save=res_save)

In [ ]:
savedir="//media//oxml//DA12F77512F754CB//Pygor_results//optimization//gamma//"

#Change xmlip to correct lab-pc, where Pygor is running
# change hardcoded normalaisation (ctrl+F <'hardcoded'>)
# change gate voltage list - will  need to cahge other stuff
#pygor = Pygor.Experiment(mode='jupyter', xmlip="http://129.67.86.107:8000/RPC2",savedir=savedir)

#window_small: Window size in mV
#res_low/res_high: Number of points in do2d

run_random(window_small=17, res_low=30, res_high=32, n_walk=3,it=150,gates_init=
[-1356.597083799452, -655.7552004185842, -757.39513943, -1910.53470352, -1315.74489394, -610.0010767257405],  starting_c5_c9_init=[-626, -782] ,savedir=savedir,res_save=128)